In [20]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
import glob
import os

CARGAR TODOS LOS ARCHIVOS Y CONCATENARLOS TODOS

In [8]:
#Hacer una lista con los archivos de la carpeta
archivos = glob.glob("/Users/nacho/Desktop/TFM/Datos/Datos equipos LaLiga J27/*xlsx")
len(archivos)

20

In [9]:
#Cargar los archivos
len(archivos)

lista_de_archivos = []
for f in archivos:
    data = pd.read_excel(f)
    nombre = os.path.basename(f)
    data['carpeta_contenedora'] = nombre
    lista_de_archivos.append(data)

In [10]:
len(lista_de_archivos)

20

In [11]:
#Concatenarlos en un solo data frame
df = pd.concat(lista_de_archivos, ignore_index= True)

In [12]:
df.shape

(1396, 105)

LIMPIAR Y ORDENAR LOS DATOS DEL ARCHIVO CONCATENADO

In [13]:
#Eliminar filas de cada partido, solo nos quedamos con las generales, las estadísticas de lo equipos y las de los rivales contra cada equipo.
df = df[~(df['Team'].str.startswith('2022') | df['Team'].str.startswith('2023'))]

In [14]:
#Crear dos df distientos uno con las estadísticas de lo equipos y otros con las estadísticas de los rivales contra cada equipo.
Opponents_stats = df[df['Team'].str.startswith('Opponents')]
Team_stats = df[~df['Team'].str.startswith('Opponents')]

2 FASE OFENSIVA CREACIÓN

In [15]:
#Renombrar columna con error
Team_stats = Team_stats.rename(columns={'Positional attacks / with shots': 'Positional attacks'})

2.1 Elección en creación

In [16]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Possession, %', ascending=True)

# Crear el gráfico
posesion = Team_stats.hvplot.barh(x='Team', y='Possession, %',
                       color='Possession, %', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Posesión', 
                       title='Porcentaje de posesión')

posesion

:Bars   [Team]   (Possession, %)

In [17]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Positional attacks', ascending=True)

# Crear el gráfico
ataques_poscionales = Team_stats.hvplot.barh(x='Team', y='Positional attacks',
                       color='Positional attacks', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Ataques posicionales por 90', 
                       title='Ataques poscionales')

ataques_poscionales

:Bars   [Team]   (Positional attacks)

In [18]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Counterattacks', ascending=True)

# Crear el gráfico
contraaques = Team_stats.hvplot.barh(x='Team', y='Counterattacks',
                       color='Counterattacks', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Contraaques por 90', 
                       title='Contraaques')

contraaques

:Bars   [Team]   (Counterattacks)

In [21]:
# Calcular las medias de las columnas.
posicionales_mean = Team_stats['% of positional attacks with shots'].mean()
contras_mean = Team_stats['% of counterattacks with shots'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
finalizacion_ataques = Team_stats.hvplot.scatter(x='% of positional attacks with shots', y='% of counterattacks with shots', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Aerial duels", "Aerial duels won"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
posicionales_mean_line = hv.VLine(posicionales_mean)
posicionales_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

contras_mean_line = hv.HLine(contras_mean)
contras_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

finalizacion_ataques = finalizacion_ataques * posicionales_mean_line * contras_mean_line.opts(title='Finalización de ataques')

finalizacion_ataques

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [% of positional attacks with shots]   (% of counterattacks with shots,color,Aerial duels,Aerial duels won)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

In [22]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Match tempo', ascending=True)

# Crear el gráfico
match_tempo = Team_stats.hvplot.barh(x='Team', y='Match tempo',
                       color='Match tempo', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Match tempo', 
                       title='Velocidad en el juego\n(Número de pases por minuto)')

match_tempo

:Bars   [Team]   (Match tempo)

In [23]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Average passes per possession', ascending=True)

# Crear el gráfico
pases_por_posesion = Team_stats.hvplot.barh(x='Team', y='Average passes per possession',
                       color='Average passes per possession', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Average passes per possession', 
                       title='Pases por posesión')

pases_por_posesion

:Bars   [Team]   (Average passes per possession)

In [24]:
#Calcular la nueva columna, Ratio de progresividad
Team_stats['Progressiveness ratio'] = Team_stats['Progressive passes'] / Team_stats['Passes']

# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Progressiveness ratio', ascending=True)

# Crear el gráfico
ratio_progresividad = Team_stats.hvplot.barh(x='Team', y='Progressiveness ratio',
                       color='Progressiveness ratio', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Progressiveness ratio', 
                       title='Ratio de progresividad')

ratio_progresividad

:Bars   [Team]   (Progressiveness ratio)

In [25]:
# Calcular las medias de las columnas.
progre_mean = Team_stats['Progressiveness ratio'].mean()
posession_mean = Team_stats['Possession, %'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
progresividad_ataques = Team_stats.hvplot.scatter(x='Progressiveness ratio', y='Possession, %', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 )

#Añadir lineas de media de goles y tiros para ver cuadrantes
progre_mean_line = hv.VLine(progre_mean)
progre_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

posession_mean_line = hv.HLine(posession_mean)
posession_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

progresividad_ataques = progresividad_ataques * progre_mean_line * posession_mean_line.opts(title='Pases')

progresividad_ataques

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Progressiveness ratio]   (Possession, %,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

2.2 Pases

In [26]:
# Calcular las medias de las columnas.
pases_mean = Team_stats['Passes'].mean()
pases_buenos_mean = Team_stats['% of Passes accurate'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
finalizacion_ataques = Team_stats.hvplot.scatter(x='Passes', y='% of Passes accurate', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Passes", "Passes accurate"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
pases_mean_line = hv.VLine(pases_mean)
pases_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

pases_buenos_mean_line = hv.HLine(pases_buenos_mean)
pases_buenos_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

total_pases = finalizacion_ataques * pases_mean_line * pases_buenos_mean_line.opts(title='Pases')

total_pases

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Passes]   (% of Passes accurate,color,Passes accurate)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

In [27]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Forward passes', ascending=True)

# Crear el gráfico
pases_delante = Team_stats.hvplot.barh(x='Team', y='Forward passes',
                       color='Forward passes', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Forward passes', 
                       title='Pases hacia delante por 90')

pases_delante

:Bars   [Team]   (Forward passes)

In [28]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Back passes', ascending=True)

# Crear el gráfico
pases_atras = Team_stats.hvplot.barh(x='Team', y='Back passes',
                       color='Back passes', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Forward passes', 
                       title='Pases hacia atrás por 90')

pases_atras

:Bars   [Team]   (Back passes)

In [29]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Lateral passes', ascending=True)

# Crear el gráfico
pases_laterales = Team_stats.hvplot.barh(x='Team', y='Lateral passes',
                       color='Lateral passes', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Forward passes', 
                       title='Pases laterales por 90')

pases_laterales

:Bars   [Team]   (Lateral passes)

In [30]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Long passes', ascending=True)

# Crear el gráfico
pases_largos = Team_stats.hvplot.barh(x='Team', y='Long passes',
                       color='Long passes', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Long passes', 
                       title='Pases largos por 90')

pases_largos

:Bars   [Team]   (Long passes)

In [31]:
# Ordenar los equipos.
Team_stats = Team_stats.sort_values('Average pass length', ascending=True)

# Crear el gráfico
distancia_pase = Team_stats.hvplot.barh(x='Team', y='Average pass length',
                       color='Average pass length', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Average pass length', 
                       title='Distancia media del pase')

distancia_pase

:Bars   [Team]   (Average pass length)

2.3 Intencionalidad de los pases

In [32]:
# Calcular las medias de las columnas.
pases_ultimo_tercio_mean = Team_stats['Passes to final third'].mean()
pases_ultimo_tercio_buenos_mean = Team_stats['% of Passes to final third accurate'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
pases_ultimo_tercio = Team_stats.hvplot.scatter(x='Passes to final third', y='% of Passes to final third accurate', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Passes to final third", "% of Passes to final third accurate"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
pases_ultimo_tercio_mean_line = hv.VLine(pases_ultimo_tercio_mean)
pases_ultimo_tercio_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

pases_ultimo_tercio_buenos_mean_line = hv.HLine(pases_ultimo_tercio_buenos_mean)
pases_ultimo_tercio_buenos_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

pases_ultimo_tercio = pases_ultimo_tercio * pases_ultimo_tercio_mean_line * pases_ultimo_tercio_buenos_mean_line.opts(title='Pases a último tercio')

pases_ultimo_tercio

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Passes to final third]   (% of Passes to final third accurate,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

In [33]:
# Calcular las medias de las columnas.
smart_mean = Team_stats['Smart passes'].mean()
smart_buenos_mean = Team_stats['% of Smart passes accurate'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
pases_smart = Team_stats.hvplot.scatter(x='Smart passes', y='% of Smart passes accurate', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Smart passes", "% of Smart passes accurate"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
smart_mean_line = hv.VLine(smart_mean)
smart_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

smart_buenos_mean_line = hv.HLine(smart_buenos_mean)
smart_buenos_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

pases_smart = pases_smart * smart_mean_line * smart_buenos_mean_line.opts(title='Pases que rompen lineas')

pases_smart

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Smart passes]   (% of Smart passes accurate,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

Guardar plots f.of. creación

In [34]:
#Guardar visualizaciones. Nombre = fase ofesniva(2)_subfase(1,2,3...)_visualización

#1. Elección de creación
hvplot.save(posesion, '2_1_posesion.png')
hvplot.save(ataques_poscionales, '2_1_ataques_poscionales.png')
hvplot.save(contraaques, '2_1_contraaques.png')
hvplot.save(finalizacion_ataques, '2_1_finalizacion_ataques.png') # porcentaje de finalización por cada tipo de ataque
hvplot.save(match_tempo, '2_1_match_tempo.png') #velocidad en el juego, número de pases por minuto
hvplot.save(pases_por_posesion, '2_1_pases_por_posesion.png')
hvplot.save(ratio_progresividad, '2_1_ratio_progresividad.png')
hvplot.save(progresividad_ataques, '2_1_progresividad_ataques.png')

#2. Pases
hvplot.save(total_pases, '2_2_total_pases.png')
hvplot.save(pases_delante, '2_2_pases_delante.png')
hvplot.save(pases_atras, '2_2_pases_atras.png')
hvplot.save(pases_laterales, '2_2_pases_laterales.png')
hvplot.save(pases_largos, '2_2_pases_largos.png')
hvplot.save(distancia_pase, '2_2_distancia_pase.png')

#3. Intencionalidad de los pases
hvplot.save(pases_ultimo_tercio, '2_3_pases_ultimo_tercio.png')
hvplot.save(pases_smart, '2_3_pases_smart.png')